In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


In [2]:
grid_data_file = os.path.join(os.getcwd().split('data')[0], 'bi.pickle')
with open(grid_data_file,'rb') as f:
    load_array= pickle.load(f)
exc = [load_array[0][20], load_array[0][50]]
inh = [load_array[1][20], load_array[1][50]]

folder = str(os.getcwd().split('data_2')[1])
print(folder)
case = str(folder[1])

file_ = 'up.pickle'

c_var = [0,1]
if len(folder) == 4:
    if folder[3] == 'e':
        p_var = [0]
    elif folder[3] == 'i':
        p_var = [1]
else:
    p_var = [0,1]
    
print(p_var)

step1 = 1
step2 = 10
step3 = 100

/1_e
[0]


In [3]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [4]:
d_array = np.arange(20., 300, 0.1)
t0_array = d_array - 20.
t_pen = np.zeros(( t0_array.shape ))
for i in range(len(t_pen)):
    t_pen[i] = t0_array[i] / d_array[i] 

In [5]:
dur_pre = 10
dur_post = 10

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-12
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

if case in ['1', '2']:    # low to high
    max_I = [3., -3.]
else:
    max_I = [-3., 3.]
    
if case in ['1', '3']:    # sparsity
    factor_ws = 1.
    factor_we = 0.
else:
    factor_ws = 0.
    factor_we = 1.
    
maxC = [1000., -1000., 0.18, 0.]
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [6]:
bestControl_ = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_ = np.array( [[None] * len(t_pen)] * len(exc) )
cost_ = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_ = np.array( [[None] * len(t_pen)] * len(exc) )
weights_ = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_ = np.array( [[None] * len(t_pen)] * len(exc) )

initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

print(bestControl_.shape)

(2, 2800)


In [7]:
if os.path.isfile(file_) :
    print("file found")
    
    with open(file_,'rb') as f:
        load_array = pickle.load(f)

    bestControl_ = load_array[0]
    bestState_ = load_array[1]
    cost_ = load_array[2]
    costnode_ = load_array[3]
    weights_ = load_array[4]

file found


In [8]:
# get initial parameters and target states
# -------  20 0.4500000000000001 0.4750000000000002
#-------  50 0.47500000000000014 0.6000000000000003

i_range = range(len(exc))
data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = 3000.
    
    control0 = aln.getZeroControl()
    control0 = functions.step_control(aln, maxI_ = max_I[0])

    aln.run(control=control0)
    
    target_rates = np.zeros((2))
    target_rates[0] = aln.rates_exc[0,-1] 
    target_rates[1] = aln.rates_inh[0,-1]

    control0 = functions.step_control(aln, maxI_ = max_I[1])
    aln.run(control=control0)

    init_state_vars = np.zeros(( len(state_vars) ))
    for j in range(len(state_vars)):
        if aln.state[state_vars[j]].size == 1:
            init_state_vars[j] = aln.state[state_vars[j]][0]
        else:
            init_state_vars[j] = aln.state[state_vars[j]][0,-1]

    initVars[i] = init_state_vars
    target[i] = [target_rates[0], target_rates[1]]

-------  0 0.4500000000000001 0.4750000000000002
-------  1 0.47500000000000014 0.6000000000000003


In [9]:
# get uncontrolled cost

data.set_parameters(aln)

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    dur = d_array[0]
    aln.params.duration = dur
    
    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]
            
    cost.setParams(1.0, 0.0, 0.0)

    setinit(initVars[i], aln)
    control0 = aln.getZeroControl()

    # "HS", "FR", "PR", "HZ"
    cgv = None
    max_it = 0

    bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])
    
    cost_uncontrolled[i] = cost_init_[0]

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 0.0
interpolate adjoint :  True True True
initial cost =  12801.807032522984
RUN  0 , total integrated cost =  12801.807032522984
Gradient descend method:  None
RUN  0 , total integrated cost =  12801.807032522984
Improved over  0  iterations in  0.0  seconds by  0.0  percent.
-------  1 0.47500000000000014 0.6000000000000003
set cost params:  1.0 0.0 0.0
interpolate adjoint :  True True True
initial cost =  16022.670213255857
RUN  0 , total integrated cost =  16022.670213255857
Gradient descend method:  None
RUN  0 , total integrated cost =  16022.670213255857
Improved over  0  iterations in  0.0  seconds by  0.0  percent.


In [10]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.

    cgv = None
    j = 0
    max_it = 10000

    while not convergence_[i][j]:

        dur = d_array[0]
        aln.params.duration = dur
        cost.setParams(1.0, 0., 1.)
        weights_[i][j] = [1., 0., 1.]
        
        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_[i][j]) == type(None) or np.amax(np.abs(bestControl_[i][j])) < 1e-6:
            control0 = aln.getZeroControl()
        else:
            control0 = bestControl_[i][j][:,:,n_pre-1:-n_post+1]

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if max_it > 1:
            if cost_[i][j][2] == 0.:
                convergence_[i][j] = True
                continue

        print('cost splitting : ', costnode_[i][j][0][0][:2], costnode_[i][j][2][0][:2])

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

-------  0 0.4500000000000001 0.4750000000000002
set cost params:  1.0 0.0 1.0
interpolate adjoint :  True True True
initial cost =  219.06639948526265
RUN  0 , total integrated cost =  219.06639948526265
Gradient descend method:  None
RUN  1 , total integrated cost =  219.06639948526265
Control only changes marginally.
RUN  1 , total integrated cost =  219.06639948526265
Improved over  1  iterations in  42.39791804552078  seconds by  0.0  percent.
-------  1 0.47500000000000014 0.6000000000000003
set cost params:  1.0 0.0 1.0
interpolate adjoint :  True True True
initial cost =  16022.670213255857
RUN  0 , total integrated cost =  16022.670213255857
Gradient descend method:  None
RUN  1 , total integrated cost =  16022.670213255857
Control only changes marginally.
RUN  1 , total integrated cost =  16022.670213255857
Improved over  1  iterations in  0.24643418192863464  seconds by  0.0  percent.


In [ ]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 10
    step = step1
    
    if j >= 10:
        step = step2
    if j >=1000:
        step = step3

    while j < 2001:
        
        if convergence_[i][j]:
            j += step
            if j >= 10:
                step = step2
            if j >=1000:
                step = step3
            continue

        print('-----', j)

        dur = round(d_array[j],1)
        aln.params.duration = dur
        max_it = int(10000)  

        setinit(initVars[i], aln)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_[i][j]) == type(None):
            control0 = aln.getZeroControl()
            control0[:,:,:-step] = bestControl_[i][j-step][:,:,n_pre-1:-n_post+1]

        else:
            control0 = bestControl_[i][j][:,:,n_pre-1:-n_post+1]
            
        weights_[i][j] = cost.getParams()

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_[i][j][-1] == 0.:
            convergence_[i][j] = True
            if j >= 10:
                step = step2
            if j >=1000:
                step = step3

        print('cost splitting : ', costnode_[i][j][0][0][:2], costnode_[i][j][2][0][:2])

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

-------  0 0.4500000000000001 0.4750000000000002
----- 260
interpolate adjoint :  True True True
initial cost =  4.92672679382854
RUN  0 , total integrated cost =  4.92672679382854
Gradient descend method:  None
RUN  1 , total integrated cost =  4.92672679382854
Control only changes marginally.
RUN  1 , total integrated cost =  4.92672679382854
Improved over  1  iterations in  0.27185237407684326  seconds by  0.0  percent.
cost splitting :  [4.99572516e-05 0.00000000e+00] [4.92661222e+00 3.84729504e-12]
----- 270
interpolate adjoint :  True True True
initial cost =  4.866940477395269
RUN  0 , total integrated cost =  4.866940477395269
Gradient descend method:  None
RUN  1 , total integrated cost =  4.866940477395269
Control only changes marginally.
RUN  1 , total integrated cost =  4.866940477395269
Improved over  1  iterations in  0.33511101454496384  seconds by  0.0  percent.
cost splitting :  [4.57905129e-05 0.00000000e+00] [4.86683318e+00 3.84729504e-12]
----- 280
interpolate adjoi

----- 440
interpolate adjoint :  True True True
initial cost =  4.24365893184327
RUN  0 , total integrated cost =  4.24365893184327
Gradient descend method:  None
RUN  1 , total integrated cost =  4.24365893184327
Control only changes marginally.
RUN  1 , total integrated cost =  4.24365893184327
Improved over  1  iterations in  0.32728657126426697  seconds by  0.0  percent.
cost splitting :  [1.05677013e-05 0.00000000e+00] [4.24362523e+00 3.84729504e-12]
----- 450
interpolate adjoint :  True True True
initial cost =  4.221632086357276
RUN  0 , total integrated cost =  4.221632086357276
Gradient descend method:  None
RUN  1 , total integrated cost =  4.221632086357276
Control only changes marginally.
RUN  1 , total integrated cost =  4.221632086357276
Improved over  1  iterations in  0.38007430732250214  seconds by  0.0  percent.
cost splitting :  [9.84626952e-06 0.00000000e+00] [4.22160020e+00 3.84729504e-12]
----- 460
interpolate adjoint :  True True True
initial cost =  4.2007627204

RUN  12 , total integrated cost =  4.01318422266505
RUN  13 , total integrated cost =  4.013184222078676
RUN  14 , total integrated cost =  4.0131842210404445
RUN  15 , total integrated cost =  4.013184220887644
RUN  16 , total integrated cost =  4.013184203979495
RUN  17 , total integrated cost =  4.013184188125288
RUN  18 , total integrated cost =  4.013184187961035
RUN  19 , total integrated cost =  4.013184187214588
RUN  20 , total integrated cost =  4.013184186885667
RUN  30 , total integrated cost =  4.013184170383697
RUN  40 , total integrated cost =  4.013184133358231
RUN  50 , total integrated cost =  4.013184125095024
RUN  60 , total integrated cost =  4.013184095133242
RUN  70 , total integrated cost =  4.013184070406907
RUN  80 , total integrated cost =  4.013184050496182
RUN  90 , total integrated cost =  4.013184017990157
RUN  100 , total integrated cost =  4.013183975945328
RUN  110 , total integrated cost =  4.013183959469692
RUN  120 , total integrated cost =  4.013183

RUN  40 , total integrated cost =  4.013145126665687
RUN  50 , total integrated cost =  4.013145121738401
RUN  60 , total integrated cost =  4.013145117464593
RUN  70 , total integrated cost =  4.013145111792774
RUN  80 , total integrated cost =  4.013145107823188
RUN  90 , total integrated cost =  4.013145103155957
RUN  100 , total integrated cost =  4.0131450978782715
RUN  110 , total integrated cost =  4.013145093909521
RUN  120 , total integrated cost =  4.013145088688881
RUN  130 , total integrated cost =  4.013145083965326
RUN  140 , total integrated cost =  4.013145079997398
RUN  150 , total integrated cost =  4.0131450747736785
RUN  160 , total integrated cost =  4.013145070053932
RUN  170 , total integrated cost =  4.013145065128285
RUN  180 , total integrated cost =  4.013145060860055
RUN  190 , total integrated cost =  4.013145055187784
RUN  200 , total integrated cost =  4.013145051219375
RUN  300 , total integrated cost =  4.013145004281184
RUN  400 , total integrated cost

RUN  140 , total integrated cost =  4.013107216093139
RUN  150 , total integrated cost =  4.013107212931575
RUN  160 , total integrated cost =  4.013107209627608
RUN  170 , total integrated cost =  4.013107206042715
RUN  180 , total integrated cost =  4.013107202796084
RUN  190 , total integrated cost =  4.013107199579023
RUN  200 , total integrated cost =  4.013107195748645
RUN  300 , total integrated cost =  4.013107162126986
RUN  400 , total integrated cost =  4.013107128705699
RUN  500 , total integrated cost =  4.013107095020315
RUN  600 , total integrated cost =  4.013107061686106
RUN  700 , total integrated cost =  4.013107028319754
RUN  800 , total integrated cost =  4.013106994509783
RUN  900 , total integrated cost =  4.013106960900734
RUN  1000 , total integrated cost =  4.013106927659114
RUN  1100 , total integrated cost =  4.013106893821089
RUN  1200 , total integrated cost =  4.013106860344271
RUN  1300 , total integrated cost =  4.013106827063666
RUN  1400 , total integr

...

RUN  19 , total integrated cost =  3.9799848805399614
RUN  20 , total integrated cost =  3.979984880512802
RUN  30 , total integrated cost =  3.9799848637593347
RUN  40 , total integrated cost =  3.9799848474312274
RUN  50 , total integrated cost =  3.9799848335359167
RUN  60 , total integrated cost =  3.9799848169255996
RUN  70 , total integrated cost =  3.979984800932335
RUN  80 , total integrated cost =  3.979984786616853
RUN  90 , total integrated cost =  3.9799847701569013
RUN  100 , total integrated cost =  3.9799847540533455
RUN  110 , total integrated cost =  3.9799847401916555
RUN  120 , total integrated cost =  3.9799847234391823
RUN  130 , total integrated cost =  3.9799847072321737
RUN  140 , total integrated cost =  3.9799846933875416
RUN  150 , total integrated cost =  3.9799846767749636
RUN  160 , total integrated cost =  3.979984660904173
RUN  170 , total integrated cost =  3.979984646640388
RUN  180 , total integrated cost =  3.9799846301744863
RUN  190 , total integra

RUN  100 , total integrated cost =  3.9799658895643635
RUN  110 , total integrated cost =  3.9799658877921824
RUN  120 , total integrated cost =  3.9799658797360906
RUN  130 , total integrated cost =  3.97996587509226
RUN  140 , total integrated cost =  3.979965864579554
RUN  150 , total integrated cost =  3.9799658573958787
RUN  160 , total integrated cost =  3.979965843979513
RUN  170 , total integrated cost =  3.9799658367883186
RUN  180 , total integrated cost =  3.9799658234506334
RUN  190 , total integrated cost =  3.9799658162648437
RUN  200 , total integrated cost =  3.979965813772101
RUN  300 , total integrated cost =  3.979965626520838
RUN  400 , total integrated cost =  3.979965499856868
RUN  500 , total integrated cost =  3.9799653013840444
RUN  600 , total integrated cost =  3.9799652045911635
RUN  700 , total integrated cost =  3.9799648742193017
RUN  800 , total integrated cost =  3.9799647157231406
RUN  900 , total integrated cost =  3.9799646220010425
RUN  1000 , total

RUN  180 , total integrated cost =  3.9799616520314847
RUN  190 , total integrated cost =  3.979961649186887
RUN  200 , total integrated cost =  3.979961644883267
RUN  300 , total integrated cost =  3.9799616076393627
RUN  400 , total integrated cost =  3.9799615953638394
RUN  500 , total integrated cost =  3.9799615855166377
RUN  600 , total integrated cost =  3.9799615756711577
RUN  700 , total integrated cost =  3.9799615658273937
RUN  800 , total integrated cost =  3.979961555985345
RUN  900 , total integrated cost =  3.979961546145016
RUN  1000 , total integrated cost =  3.9799615337889866
RUN  1100 , total integrated cost =  3.9799615239521975
RUN  1200 , total integrated cost =  3.9799615141171305
RUN  1300 , total integrated cost =  3.9799615042837795
RUN  1400 , total integrated cost =  3.979961494452146
RUN  1500 , total integrated cost =  3.9799614663510945
RUN  1600 , total integrated cost =  3.979961434633142
RUN  1700 , total integrated cost =  3.979961401394008
RUN  1800

RUN  900 , total integrated cost =  3.979949070077401
RUN  1000 , total integrated cost =  3.979949061237002
RUN  1100 , total integrated cost =  3.979949052397698
RUN  1200 , total integrated cost =  3.9799490435594924
RUN  1300 , total integrated cost =  3.9799490347223845
RUN  1400 , total integrated cost =  3.9799490258863712
RUN  1500 , total integrated cost =  3.979949017051459
RUN  1600 , total integrated cost =  3.9799490082176425
RUN  1700 , total integrated cost =  3.979948999384926
RUN  1800 , total integrated cost =  3.9799489905533076
RUN  1900 , total integrated cost =  3.9799489817227927
RUN  2000 , total integrated cost =  3.9799489729609245
RUN  3000 , total integrated cost =  3.9799488847271665
RUN  4000 , total integrated cost =  3.979948732493848
RUN  5000 , total integrated cost =  3.9799486445592787
RUN  6000 , total integrated cost =  3.979948556735696
RUN  7000 , total integrated cost =  3.9799484654976793
RUN  8000 , total integrated cost =  3.9799483597659044


RUN  1700 , total integrated cost =  3.9799423897924835
RUN  1800 , total integrated cost =  3.979942381926524
RUN  1900 , total integrated cost =  3.9799423740642035
RUN  2000 , total integrated cost =  3.979942366205528
RUN  3000 , total integrated cost =  3.979942283831909
RUN  4000 , total integrated cost =  3.979942201673026
RUN  5000 , total integrated cost =  3.979941994639307
RUN  6000 , total integrated cost =  3.979941917969465
RUN  7000 , total integrated cost =  3.979941841654141
RUN  8000 , total integrated cost =  3.979941622583678
RUN  9000 , total integrated cost =  3.979941547635412


In [ ]:
bestControl_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
cost_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
weights_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_maxW = np.array( [[None] * len(t_pen)] * len(exc) )

file_maxW = '1_up_Wmax.pickle'

In [ ]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0

    while not convergence_maxW[i][j]:

        dur = d_array[0]
        aln.params.duration = dur        
        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        if type(bestControl_maxW[i][j]) == type(None):
            max_it = 10
            control0 = bestControl_[i][0][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] - sum(costnode_[i][j][0][0][:]) ) / sum( costnode_[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            max_it = int(100)

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_[i][j][0][0][:2], costnode_[i][j][2][0][:2])

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

In [ ]:
for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 1
    step = step1

    while j < 2001:
        
        if j >= 1000:
            step = step2
        
        if convergence_maxW[i][j]:
            j += step
            continue

        print('-----', j)

        dur = round(d_array[j],1)
        aln.params.duration = dur
        max_it = int(10000)  

        setinit(initVars[i], aln)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_maxW[i][j]) == type(None):
            control0 = aln.getZeroControl()
            control0[:,:,:-step] = bestControl_maxW[i][j-step][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                          - sum(costnode_maxW[i][j-step][0][0][:]) ) / sum( costnode_maxW[i][j-step][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()

        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i] * (1. - t_pen[0]) / (1. - t_pen[j])
                           - sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            cost.setParams(1., weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_maxW[i][j][0][0][:2], costnode_maxW[i][j][2][0][:2])

    with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)          